In [ ]:
'extract abnormal spatial locations and all z-scores on Sep.1-Sep.2 (Ida day) & Aug.21-22(Henri day)'
import pandas as pd
import os
import numpy as np

def classify(df):
    cut_labels = ['Regular', 'Potential Irregular (+)', 'Irregular (+)']
    cut_bins = [0, 2.5, 3, np.inf]
    df['class'] = pd.cut(df['abs_ZS_new'], bins=cut_bins, labels=cut_labels)
    df["class"] = df["class"].astype("str") 
    s1 = df[(df['class']=='Potential Irregular (+)')&(df['norm_resid']<0)].index
    if len(s1)>0:
        df.loc[s1,'class'] = 'Potential Irregular (-)'
    s2 = df[(df['class']=='Irregular (+)')&(df['norm_resid']<0)].index
    if len(s2)>0:        
        df.loc[s2,'class'] = 'Irregular (-)'
    # s3 = df[(df['class']=='Highly Irregular (+)')&(df['resid']<0)].index
    # if len(s3)>0:
    #     df.loc[s3,'class'] = 'Highly Irregular (-)'
    
    
    return df

def col_sel(df, col, dt, nm):
    df_sel = df[df['ds'].str[0:10]==dt][[col,'ds','abs_ZS_new','zscore','norm_resid']]
    df_sel = classify(df_sel)
    df_sel = df_sel.rename(columns={'abs_ZS_new':'az'+nm,'zscore':'z'+nm,'norm_resid':'r'+nm,'class':'c'+nm})
    return df_sel

def run(dir,file):
    df = pd.read_csv(dir+file)
    col = df.columns[-2]
    print(file)
    if 'TLC' in dir:
        df[col] = df[col].str.split(".",expand=True)[0]
        df[col] = df[col].str[1:]#change taxi zone id to related to shpfile
        
    if 'Turnstile' in dir:
        col = df.columns[-1]
    print(col)
    dfz = df[df['abs_ZS_new']>=3]
    
    sta = dfz.groupby([col,'ds']).sum()[['abs_ZS_new','zscore']]
    sta.reset_index(inplace=True)
    print(sta)
    sta.to_csv(dir+'outlier_loc_'+file,index=False)#save all outliers' zscore
    
    
    Sep1 = col_sel(df, col, '2021-09-01', '_210901')
    Sep2 = col_sel(df, col, '2021-09-02', '_210902')
    
    Aug1 = col_sel(df, col, '2021-08-21', '_210821')
    Aug2 = col_sel(df, col, '2021-08-22', '_210822')
    Aug3 = col_sel(df, col, '2021-08-23', '_210823')
    
    Jul1 = col_sel(df, col, '2021-07-08', '_210708')
    Jul2 = col_sel(df, col, '2021-07-09', '_210709')
    
    Oct1 = col_sel(df, col, '2021-10-25', '_211025')
    Oct2 = col_sel(df, col, '2021-10-26', '_211026')
    
    
    
    if 'Hourly' in file:
        Sep1['hour'] = Sep1['ds'].str[11:13]
        Sep2['hour'] = Sep2['ds'].str[11:13]
        
        Aug1['hour'] = Aug1['ds'].str[11:13]
        Aug2['hour'] = Aug2['ds'].str[11:13]
        Aug3['hour'] = Aug3['ds'].str[11:13]
        
        Jul1['hour'] = Jul1['ds'].str[11:13]
        Jul2['hour'] = Jul2['ds'].str[11:13]
        
        Oct1['hour'] = Oct1['ds'].str[11:13]
        Oct2['hour'] = Oct2['ds'].str[11:13]
        
        
        
        for i in ['00','04','08','12','16','20']:
            t1 = Sep1[Sep1['hour']==i]
            t2 = Sep2[Sep2['hour']==i]
            Sep =pd.merge(t1,t2,on = col,how="outer")
            Sep.to_csv(dir+'Sep_'+i+'_'+file,index=False)#save all zscore on Sep.1-2.
            
            t3 = Aug1[Aug1['hour']==i]
            t4 = Aug2[Aug2['hour']==i]
            t5 = Aug3[Aug3['hour']==i]
            tt =pd.merge(t3,t4,on = col,how="outer")
            Aug =pd.merge(tt,t5,on = col,how="outer")
            Aug.to_csv(dir+'Aug_'+i+'_'+file,index=False)#save all zscore on Aug.21-22.
            
            t6 = Jul1[Jul1['hour']==i]
            t7 = Jul2[Jul2['hour']==i]
            Jul =pd.merge(t6,t7,on = col,how="outer")
            Jul.to_csv(dir+'Jul_'+i+'_'+file,index=False)#save all zscore on Jul.7-8.
            
            t8 = Oct1[Oct1['hour']==i]
            t9 = Oct2[Oct2['hour']==i]
            Oct =pd.merge(t8,t9,on = col,how="outer")
            Oct.to_csv(dir+'Oct_'+i+'_'+file,index=False)#save all zscore on Oct.25-26.
            

    else:
        # col = [col , 'hour']
    
        Sep =pd.merge(Sep1,Sep2,on = col,how="outer")
        Sep.to_csv(dir+'Sep_'+file,index=False)#save all zscore on Sep.1-2.
        
        Aug =pd.merge(Aug1,Aug2,on = col,how="outer")
        Aug.to_csv(dir+'Aug_'+file,index=False)#save all zscore on Aug.21-22.
        
        Jul =pd.merge(Jul1,Jul2,on = col,how="outer")
        Jul.to_csv(dir+'Jul_'+file,index=False)#save all zscore on Jul.7-8.
        
        Oct =pd.merge(Oct1,Oct2,on = col,how="outer")
        Oct.to_csv(dir+'Oct_'+file,index=False)#save all zscore on Oct.25-26.


if __name__=='__main__':
    
    main_dir = r""
    dirs = [r'\TLC\result\without holiday/',r'\Turnstile\result\without holiday/',r'\citibike\result\without holiday/']
    for dir in dirs:
        dir = main_dir + dir
        items = [i for i in os.listdir(dir) if os.path.splitext(i)[1] == '.csv']
        for file in items:
            if ('Sep' in file) | ('Aug' in file) | ('Jul' in file) | ('Oct' in file) | ('outlier' in file):
                continue
            run(dir,file)
            
        
